In [1]:
import joblib
import pandas as pd
import numpy as np

In [2]:
%%writefile property_server.py

from flask import Flask, request, jsonify
import joblib
import pandas as pd
import numpy as np

app = Flask(__name__)


model = joblib.load("property_v0.model") #wczytaj wytrenowany model


def fe(df):

    df["area_num"] = df["area"].map(lambda x: x.split("m")[0].replace(" ", "").replace(",", ".")).astype("float")
    df["area_num_log"] = np.log(df["area_num"])
    area_num_99 = np.percentile(df["area_num"], 99)
    df["area_norm"] = df["area_num"].map(lambda x: x if x <= area_num_99 else area_num_99  )
    df["area_per_room"] = df["area_norm"] / df["rooms"]
    

    for i in range(5):
        df["loc{}".format(i)] = df["location"].map(lambda x: x[i] if len(x) > i else "")

    df['loc01'] = df['loc0'] + df['loc1']
    df['loc012'] = df['loc0'] + df['loc1'] + df['loc2']
    df['loc12'] = df['loc1'] + df['loc2']

    citi_areas = ['PoznańWinogrady', 'WarszawaŚródmieście', 'KrakówKrowodrza', 'WarszawaBiałołęka', 'KrakówStare Miasto', 'kołobrzeskiKołobrzeg', 'WarszawaMokotów', 'ŁódźŚródmieście', 'PoznańGrunwald', 'WrocławKrzyki']
    for item in citi_areas:
        df[item] = df["loc12"] == item


    big_cities = {'Poznań', 'Sopot', 'Wrocław', 'Kraków', 'Gdańsk', 'Gdynia', 'Opole', 'Katowice',  'Częstochowa', 'Szczecin', 'Kalisz', 'Łódź', 'Olsztyn', 'Warszawa'}
    for city in big_cities:
        df[city] = df["loc1"] == city
        df["big_city"] = df["loc1"].map(lambda x: x in big_cities)


    return df


@app.route('/predict', methods=["POST"])
def get_forecast():

    try:
        print(request.json)
        df = pd.DataFrame(request.json)
        print(df)
        if "location" in df:
            df["location"] = df["location"].map(lambda x: x.split(","))


        df_fe = fe(df)
        feats = ['area_num', 'area_num_log', 'area_per_room', 'rooms', 'big_city', 'PoznańWinogrady', 'WarszawaŚródmieście', 'KrakówKrowodrza', 'WarszawaBiałołęka', 'KrakówStare Miasto', 'kołobrzeskiKołobrzeg', 'WarszawaMokotów', 'ŁódźŚródmieście', 'PoznańGrunwald', 'WrocławKrzyki', 'Częstochowa', 'Szczecin', 'Kraków', 'Gdańsk', 'Poznań', 'Gdynia', 'Opole', 'Olsztyn', 'Sopot', 'Wrocław', 'Łódź', 'Warszawa', 'Kalisz', 'Katowice']
        X = df_fe[feats].values
        y_pred = model.predict(X)

        return jsonify(prices=[float(x) for x in y_pred], status="ok")
    except Exception as e:
        print(e)
        return jsonify(message="something is going wrong", status="error")


if __name__ == '__main__':
      app.run(host='0.0.0.0', port=8051)

Overwriting property_server.py


## Excel

Tu jest >> [szablon](https://docs.google.com/spreadsheets/d/1UYOodLl6nV4T3jVDAEr8zHSTDfozHkVJ1Lngzodo2sI/edit?usp=sharing) <<. Kliknij tam i zrób kopię (klikając w menu `File > make copy`). To daje Ci własną kopią, w której możesz robić co chcesz (w tym także udostępniać innym).

### App Scripts

Aby móc komunikować się z Flaskiem, będziemy używać `App Scripts`.Żeby się tam dostać trzeba kliknąć w menu `Extensions > App Scripts`.

Kod piszemy w javascript.

![](../images/app_scripts0.jpg)

## Pozwolenia

Na początek trzeba udzielić pozowolenia na robienie zapytań do flaska oraz pozwolenia na dostęp do zawartości Excela. 

Poniżej masz przykład jak można to zrobić.

![](../images/app_scripts1.jpg)
![](../images/app_scripts2.jpg)
![](../images/app_scripts3.jpg)

## Konfiguracja 

W `App Scripts` na samej górze masz dwie zmienne do konfiguracji


```
var urlToPredict = "https://club.dataworkshop.eu/user/ula/proxy/8051/predict";
var jhub_final_cookie = ""
```



Piewsze to link do Flask. Jak kliknisz tutaj 👉👉 [na link](/hub/user-redirect/proxy/8051/predict) 👈👈 to dostaniesz ten link. Po prostu go skopiuj, on będzie podobny do tego powyżej (zmieni się tylko Twój `username`).




Drugie, to cookie z `jupytera`. To trzeba skopiować będąc w tej zakładce. Wystarczy, że klikniesz prawy `command + option + C` na Macu lub `Control + Shift + C` na windows (w Chrome), to wtedy otworzy się konsola i tam przełączasz się do zakładki `Network` i odświeżasz stronę.




Szukasz żądanie, które zawiera `day2` i kopiujesz zawartość cookie (wystarczy tylko początek). Zwróć uwagę, że samego słowa `Cookie` kopiować nie trzeba. To co skopiujesz przypisujesz do zmiennej `jhub_final_cookie` pomiędzy cudzysłowami.



![](../images/cookie.jpg)



Dzięki temu już możesz uruchamiać. Można to zrobić na dwa sposoby:

1. wewnątrz `App Scripts`, wystarczy kliknąć `Run`

2. wracając do Excela, w menu pojawiło się w menu `Custom Menu` - kliknij tam i zobaczysz `make prediction`, klikasz, czekasz chwilę i jeśli wszystko poszło dobrze to zobaczysz wynik.🎉




### Pamiętaj, że Twój Flask musi być uruchomiony (poniżej jest polecenie) 👇. Udanej zabawy. 🥳

In [4]:
!python property_server.py

 * Serving Flask app "property_server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://0.0.0.0:8051/ (Press CTRL+C to quit)
[{'area': '104.23m2', 'rooms': 5, 'location': 'małopolskie,Kraków,Bronowice'}, {'area': '56.23m2', 'rooms': 3, 'location': 'małopolskie,Kraków,Bronowice'}, {'area': '18,23m2', 'rooms': 1, 'location': 'małopolskie,Kraków,Bronowice'}, {'area': '55,11m2', 'rooms': 2, 'location': 'małopolskie,Kraków,Bronowice'}, {'area': '55,11m2', 'rooms': 2, 'location': 'dolnośląskie,Wrocław'}, {'area': '55,11m2', 'rooms': 2, 'location': 'pomorskie,Gdynia,Chylonia,Starogardzka'}, {'area': '55,11m2', 'rooms': 2, 'location': 'wilekopolska,Poznań,Wilczak'}, {'area': '51,51m2', 'rooms': 2, 'location': 'wilekopolska,Poznań,Piatkowo'}]
       area  rooms                                location
0  104.23m2      5            małopolskie,Kraków,Bronowice
1   56.23m2      3            małopolskie,Kraków,Bronowice
2   1